In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


In [0]:
cd /content/drive/My Drive/Datathon Interbank

/content/drive/My Drive/Datathon Interbank


In [0]:
import pandas as pd
from sklearn.preprocessing import Imputer
from collections import Counter
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

In [0]:
#Leemos los datos
train_bd = pd.read_csv('ib_base_inicial_train.csv', parse_dates=['codmes'], infer_datetime_format=True)
test_bd = pd.read_csv('ib_base_inicial_test.csv', parse_dates=['codmes'], infer_datetime_format=True)
#campanias_bd = pd.read_csv('ib_base_campanias.csv', parse_dates=['codmes'], infer_datetime_format=True)
digital_bd = pd.read_csv('ib_base_digital.csv', parse_dates=['codday'], infer_datetime_format=True)
rcc_bd = pd.read_csv('ib_base_rcc.csv', parse_dates=['codmes'], infer_datetime_format=True)
reniec_bd = pd.read_csv('ib_base_reniec.csv')
sunat_bd = pd.read_csv('ib_base_sunat.csv')
#vehicular_bd = pd.read_csv('ib_base_vehicular.csv')
#sample_sub = pd.read_csv('sampleSubmission.csv')
'''Indicamos parse_dates e infer_datetime_format para que las dechas no sean integer 
ya que originalmente las columnas de fecha están con el formato 201907'''

'Indicamos parse_dates e infer_datetime_format para que las dechas no sean integer \nya que originalmente las columnas de fecha están con el formato 201907'

In [0]:
digital_bd = pd.read_csv('ib_base_digital.csv', parse_dates=['codday'], infer_datetime_format=True)
rcc_bd = pd.read_csv('ib_base_rcc.csv', parse_dates=['codmes'], infer_datetime_format=True)

#Data preparation



To do:
- Feature engineering
- escalamiento de variables numéricas

## Dataframes con codigomes

### digital_db

In [0]:
#Eliminamos las filas duplicadas digital_db
digital_bd = digital_bd.drop_duplicates()
print('shape: {}'.format(digital_bd.shape))
print('id_persona duplicados: {}'.format(digital_bd.id_persona.duplicated().sum()))
print('filas duplicadas: {}'.format(digital_bd.duplicated().sum()))

shape: (60279, 33)
id_persona duplicados: 37187
filas duplicadas: 0


In [0]:
#Creando la columna codmes
def crear_codmes(x):
  x = x.strftime('%Y-%m-%d')
  b = x.split('-')[0]
  c = x.split('-')[1]
  return b + c
digital_bd['codmes'] = digital_bd.codday.apply(crear_codmes)
digital_bd.codmes.head() 

0    201812
1    201811
2    201812
3    201811
4    201904
Name: codmes, dtype: object

In [0]:
#Creando columna prediction_id
digital_bd['prediction_id'] = digital_bd.id_persona.apply(str) + '_' + digital_bd.codmes.apply(str)
digital_bd.head(2)

,codday,id_persona,simu_prestamo,benefit,email,facebook,goog,youtb,compb,movil,desktop,n_rep30,recencia,lima_dig,provincia_dig,extranjero_dig,time_ctasimple,time_mllp,time_mllst,time_ctasld,time_tc,n_sesion,busqtc,busqvisa,busqamex,busqmc,busqcsimp,busqmill,busqcsld,busq,n_pag,android,iphone,codmes,prediction_id
0,2018-12-04,99258,0.0,0,0,1,0,0,0,1,0,1,1.0,0,1,0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,2,8.0,NaN,201812,99258_201812
1,2018-11-02,99258,0.0,0,0,1,0,0,0,1,0,0,100.0,1,0,0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,2,8.0,NaN,201811,99258_201811


In [0]:
#Agrupamos sumando los datos de "número de veces" y "tiempo"
lista = ['simu_prestamo', 'benefit', 'email', 'facebook', 'goog', 'youtb', 'compb', 
         'movil', 'desktop','n_sesion', 'n_pag', 'time_ctasimple']
sdf_digit_num = digital_bd.groupby('prediction_id')[lista].sum().reset_index()
sdf_digit_num.head()

,prediction_id,simu_prestamo,benefit,email,facebook,goog,youtb,compb,movil,desktop,n_sesion,n_pag,time_ctasimple
0,100003_201903,0.0,0,0,2,0,0,0,2,0,2,4,0.0
1,100014_201901,0.0,0,0,0,10,0,0,0,24,24,61,0.0
2,100031_201812,0.0,0,0,0,1,0,0,0,1,1,1,0.0
3,100050_201811,0.0,0,0,1,0,0,0,0,1,1,1,0.0
4,100050_201901,0.0,0,1,0,0,0,0,0,1,1,8,0.0


In [0]:
#Agrupamos sumando los datos de variables binarias cuyos valores son 0 y 1
lista = ['busqtc', 'busqvisa', 'busqcsimp', 'busqmill', 'busqcsld', 'busq',
         'n_rep30', 'lima_dig', 'provincia_dig', 'extranjero_dig']
sdf_digit_bin = digital_bd.groupby('prediction_id')[lista].sum().reset_index()
sdf_digit_bin.head()

,prediction_id,busqtc,busqvisa,busqcsimp,busqmill,busqcsld,busq,n_rep30,lima_dig,provincia_dig,extranjero_dig
0,100003_201903,0,0,0,0,0,0,1,1,1,0
1,100014_201901,0,0,0,0,0,0,1,8,0,0
2,100031_201812,0,0,0,0,0,0,0,0,1,0
3,100050_201811,0,0,0,0,0,0,1,1,0,0
4,100050_201901,0,0,0,0,0,0,0,1,0,0


In [0]:
#Agrupamos sumando los datos de variables categoricas
#Se omite la columna android y recencia, solo se usa la columna android
sdf_digit_cat = digital_bd.groupby('prediction_id')['iphone'].count().reset_index()
sdf_digit_cat.iphone.value_counts()

0     29823
1      1522
2       428
3       162
4        47
5        32
6        12
7         8
8         6
12        1
11        1
10        1
9         1
Name: iphone, dtype: int64

In [0]:
#Haciendo merge a sdfs
sdfs = [sdf_digit_num, sdf_digit_bin, sdf_digit_cat] 
digital_bdc = reduce(lambda right, left: pd.merge(right, left, on='prediction_id', how='outer'), sdfs)

In [0]:
#Creamos codmes e id_persona
digital_bdc['id_persona'] = digital_bdc.prediction_id
digital_bdc['codmes'] = digital_bdc.prediction_id

def crear_codmes(x):
    return int(x.split('_')[1])

def crear_idpersona(x):
  return int(x.split('_')[0])

digital_bdc['codmes'] = digital_bdc.codmes.apply(crear_codmes)
digital_bdc['id_persona'] = digital_bdc.id_persona.apply(crear_idpersona)
digital_bdc.codmes = digital_bdc.codmes.astype(str)
digital_bdc.id_persona = digital_bdc.id_persona.astype(str)

In [0]:
digital_bdc.head()

,prediction_id,simu_prestamo,benefit,email,facebook,goog,youtb,compb,movil,desktop,n_sesion,n_pag,time_ctasimple,busqtc,busqvisa,busqcsimp,busqmill,busqcsld,busq,n_rep30,lima_dig,provincia_dig,extranjero_dig,iphone,id_persona,codmes
0,100003_201903,0.0,0,0,2,0,0,0,2,0,2,4,0.0,0,0,0,0,0,0,1,1,1,0,0,100003,201903
1,100014_201901,0.0,0,0,0,10,0,0,0,24,24,61,0.0,0,0,0,0,0,0,1,8,0,0,0,100014,201901
2,100031_201812,0.0,0,0,0,1,0,0,0,1,1,1,0.0,0,0,0,0,0,0,0,0,1,0,0,100031,201812
3,100050_201811,0.0,0,0,1,0,0,0,0,1,1,1,0.0,0,0,0,0,0,0,1,1,0,0,0,100050,201811
4,100050_201901,0.0,0,1,0,0,0,0,0,1,1,8,0.0,0,0,0,0,0,0,0,1,0,0,0,100050,201901


In [0]:
#Exportamos digital_bdc
digital_bdc.to_csv('digital_bdc.csv', index=False)

### train_bd

In [0]:
'''
#Cambiamos formato de codmes 
train_bd['codmes'] = pd.to_datetime(train_bd.codmes, format='%Y%m')
train_bd.codmes.head()
'''

###test_db (falta)

In [0]:
'''
#Damos formato a columna fecha
test_bd['codmes'] = pd.to_datetime(test_bd.codmes, format='%Y%m')
test_bd.codmes.head()
'''

###rcc_bd (tiene codmes e id_persona)

In [0]:
#Drop a la columna clasif
rcc_bd = rcc_bd.drop('clasif', axis=1)

In [0]:
#Creando columna prediction_id
rcc_bd['prediction_id'] = rcc_bd.id_persona.apply(str) + '_' + rcc_bd.codmes.apply(str)
rcc_bd.head(2)

,codmes,id_persona,cod_banco,producto,mto_saldo,rango_mora,prediction_id
0,201811,10155,29,PRESTAMOS COMERCIALES,36698.4870,1.0,10155_201811
1,201804,75442,29,PRESTAMO PERSONAL,7971.1182,1.0,75442_201804


In [0]:
'''
#Cambiamos formato de codmes 
rcc_bd['codmes'] = pd.to_datetime(rcc_bd.codmes, format='%Y%m')
rcc_bd.codmes.head()
'''

####Tratando los missing values

In [0]:
'''
#Detectamos si hay missing values
rcc_bd.isnull().sum()
'''

codmes                 0
id_persona             0
cod_banco              0
producto               0
mto_saldo              0
rango_mora       4850443
prediction_id          0
dtype: int64

In [0]:
'''
rcc_bd[rcc_bd.producto == 'LINEA TOTAL TC'].isnull().sum()
'''

In [0]:
#llenando los missing values en rango_mora
rcc_bd.rango_mora.fillna(0, inplace=True)
rcc_bd['rango_mora'] = rcc_bd.rango_mora.astype(int)
rcc_bd.head(2)

,codmes,id_persona,cod_banco,producto,mto_saldo,rango_mora,prediction_id
0,201811,10155,29,PRESTAMOS COMERCIALES,36698.4870,1,10155_201811
1,201804,75442,29,PRESTAMO PERSONAL,7971.1182,1,75442_201804


Podemos ver que hay valores nulos solo en las columnas clasif y rango_mora, coincidentemente en las columnas en que figura LINEA TOTAL TC de la columna producto. Tiene sentido ya que no existe mora en un línea de crédito. Se debe separar la columna LINEA TOTAL TC del dataframe

#### Creando nuevos dataframes

##### Productos, mto saldo y rango_mora

In [0]:
#Creando df sdf_rcc_lineaTC_bd
sdf_rcc_lineaTC = rcc_bd[rcc_bd.producto == 'LINEA TOTAL TC']
sdf_rcc_lineaTC = sdf_rcc_lineaTC.groupby('prediction_id')['mto_saldo'].sum().reset_index()
sdf_rcc_lineaTC.rename(columns={'mto_saldo':'LINEA_TOTAL_TC'}, inplace=True)
sdf_rcc_lineaTC.head(1)

,prediction_id,LINEA_TOTAL_TC
0,100000_201712,4343.5


In [0]:
#Creando df sdf_rcc_tarj_compras
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_tarj_compras = rcc_bd[rcc_bd.producto == 'TARJETAS COMPRAS']
sdf_rcc_tarj_compras = sdf_rcc_tarj_compras.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_tarj_compras.rename(columns={'mto_saldo':'TARJETAS_COMPRAS','rango_mora':'MORA_TARJETAS_COMPRAS'}, inplace=True)
sdf_rcc_tarj_compras['MORA_TARJETAS_COMPRAS'] = sdf_rcc_tarj_compras.MORA_TARJETAS_COMPRAS.astype(int)
sdf_rcc_tarj_compras.head(1)

,prediction_id,TARJETAS_COMPRAS,MORA_TARJETAS_COMPRAS
0,100000_201712,932.794,1


In [0]:
#Creando df sdf_rcc_prest_pers
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_prest_pers = rcc_bd[rcc_bd.producto == 'PRESTAMO PERSONAL']
sdf_rcc_prest_pers = sdf_rcc_prest_pers.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_prest_pers.rename(columns={'mto_saldo':'PRESTAMO_PERSONAL', 'rango_mora':'MORA_PRESTAMO_PERSONAL'}, inplace=True)
sdf_rcc_prest_pers['MORA_PRESTAMO_PERSONAL'] = sdf_rcc_prest_pers.MORA_PRESTAMO_PERSONAL.astype(int)
sdf_rcc_prest_pers.head(1)

,prediction_id,PRESTAMO_PERSONAL,MORA_PRESTAMO_PERSONAL
0,100000_201712,1120.5719,1


In [0]:
#Creando df sdf_rcc_tarj_efect
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_tarj_efect = rcc_bd[rcc_bd.producto == 'TARJETAS EFECTIVO']
sdf_rcc_tarj_efect = sdf_rcc_tarj_efect.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_tarj_efect.rename(columns={'mto_saldo':'TARJETAS_EFECTIVO', 'rango_mora':'MORA_TARJETAS_EFECTIVO'}, inplace=True)
sdf_rcc_tarj_efect['MORA_TARJETAS_EFECTIVO'] = sdf_rcc_tarj_efect.MORA_TARJETAS_EFECTIVO.astype(int)
sdf_rcc_tarj_efect.head(1)

,prediction_id,TARJETAS_EFECTIVO,MORA_TARJETAS_EFECTIVO
0,100000_201712,421.6407,1


In [0]:
#Creando df sdf_rcc_prest_comer
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_prest_comer = rcc_bd[rcc_bd.producto == 'PRESTAMOS COMERCIALES']
sdf_rcc_prest_comer = sdf_rcc_prest_comer.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_prest_comer.rename(columns={'mto_saldo':'PRESTAMOS_COMERCIALES', 'rango_mora':'MORA_PRESTAMOS_COMERCIALES'}, inplace=True)
sdf_rcc_prest_comer['MORA_PRESTAMOS_COMERCIALES'] = sdf_rcc_prest_comer.MORA_PRESTAMOS_COMERCIALES.astype(int)
sdf_rcc_prest_comer.head(1)

,prediction_id,PRESTAMOS_COMERCIALES,MORA_PRESTAMOS_COMERCIALES
0,100002_201803,3075.8988,1


In [0]:
#Creando df sdf_rcc_tarj_otros_prod
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_tarj_otros_prod = rcc_bd[rcc_bd.producto == 'TARJETAS OTROS CONCEPTOS']
sdf_rcc_tarj_otros_prod = sdf_rcc_tarj_otros_prod.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_tarj_otros_prod.rename(columns={'mto_saldo':'TARJ_OTROS_CONCEPTOS', 'rango_mora':'MORA_TARJ_OTROS_CONCEPTOS'}, inplace=True)
sdf_rcc_tarj_otros_prod['MORA_TARJ_OTROS_CONCEPTOS'] = sdf_rcc_tarj_otros_prod.MORA_TARJ_OTROS_CONCEPTOS.astype(int)
sdf_rcc_tarj_otros_prod.head(1)                 

,prediction_id,TARJ_OTROS_CONCEPTOS,MORA_TARJ_OTROS_CONCEPTOS
0,100005_201805,0.0365,1


In [0]:
#Creando df sdf_rcc_produc_irrelev
prod_imp = ['LINEA TOTAL TC', 'TARJETAS COMPRAS', 'PRESTAMO PERSONAL', 
            'TARJETAS EFECTIVO', 'PRESTAMOS COMERCIALES', 'TARJETAS OTROS CONCEPTOS']
sdf_rcc_produc_irrelev = rcc_bd[~ rcc_bd.producto.isin(prod_imp)]
metodos = {'mto_saldo':'sum','rango_mora':'mean'}
sdf_rcc_produc_irrelev = sdf_rcc_produc_irrelev.groupby('prediction_id').agg(metodos).reset_index()
sdf_rcc_produc_irrelev.rename(columns={'mto_saldo':'PRODUCTOS_BAJO_IMPACTO', 'rango_mora':'MORA_PROD_BAJA_IMP'}, inplace=True)
sdf_rcc_produc_irrelev['MORA_PROD_BAJA_IMP'] = sdf_rcc_produc_irrelev.MORA_PROD_BAJA_IMP.astype(int) 
sdf_rcc_produc_irrelev.head(1)      

,prediction_id,PRODUCTOS_BAJO_IMPACTO,MORA_PROD_BAJA_IMP
0,100023_201712,185026.7928,1


##### cod_banco

In [0]:
sdf_rcc_codb = rcc_bd.groupby('prediction_id')['cod_banco'].count().reset_index()
sdf_rcc_codb.head(2)

,prediction_id,cod_banco
0,100000_201712,5
1,100000_201801,5


##### merge a sdfs

In [0]:
sdfs = [sdf_rcc_lineaTC, sdf_rcc_prest_comer, sdf_rcc_prest_pers, sdf_rcc_produc_irrelev,
 sdf_rcc_tarj_compras, sdf_rcc_tarj_efect, sdf_rcc_tarj_otros_prod, sdf_rcc_codb]
rcc_bd2 = reduce(lambda left,right: pd.merge(left,right,on='prediction_id', how='outer'), sdfs)
''' inputamos missin values con el valor 0 ya que esas personas, en ese
 periodo no tuvieron deuda en esos productos
 '''

rcc_bd2.fillna(0, inplace=True)
rcc_bd2['id_persona'] = rcc_bd2.prediction_id
rcc_bd2['codmes'] = rcc_bd2.prediction_id

def crear_codmes(x):
    return int(x.split('_')[1])

def crear_idpersona(x):
  return int(x.split('_')[0])


rcc_bd2['codmes'] = rcc_bd2.codmes.apply(crear_codmes)
rcc_bd2['id_persona'] = rcc_bd2.id_persona.apply(crear_idpersona)
rcc_bd2.codmes = rcc_bd2.codmes.astype(str)
rcc_bd2.id_persona = rcc_bd2.id_persona.astype(str)

In [0]:
rcc_bd2.head()

,prediction_id,LINEA_TOTAL_TC,PRESTAMOS_COMERCIALES,MORA_PRESTAMOS_COMERCIALES,PRESTAMO_PERSONAL,MORA_PRESTAMO_PERSONAL,PRODUCTOS_BAJO_IMPACTO,MORA_PROD_BAJA_IMP,TARJETAS_COMPRAS,MORA_TARJETAS_COMPRAS,TARJETAS_EFECTIVO,MORA_TARJETAS_EFECTIVO,TARJ_OTROS_CONCEPTOS,MORA_TARJ_OTROS_CONCEPTOS,cod_banco,id_persona,codmes
0,100000_201712,4343.5,0.0,0.0,1120.5719,1.0,0.0,0.0,932.7940,1.0,421.6407,1.0,0.0,0.0,5,100000,201712
1,100000_201801,4343.5,0.0,0.0,1057.2882,1.0,0.0,0.0,818.0964,1.0,211.4080,1.0,0.0,0.0,5,100000,201801
2,100000_201802,4343.5,0.0,0.0,993.0263,1.0,0.0,0.0,682.9588,1.0,143.1676,1.0,0.0,0.0,5,100000,201802
3,100000_201803,4343.5,0.0,0.0,922.8222,1.0,0.0,0.0,569.1372,1.0,72.4379,1.0,0.0,0.0,5,100000,201803
4,100000_201804,4343.5,0.0,0.0,853.6255,1.0,0.0,0.0,455.3156,1.0,0.0000,0.0,0.0,0.0,4,100000,201804


In [0]:
'''
rcc_bd2.isnull().sum()
'''

In [0]:
print('prediction_id duplicados: {}'.format(rcc_bd2.prediction_id.duplicated().sum()))

In [0]:
rcc_bd2.to_csv('rcc_bd2.csv', index=False)

###Campanias_bd (Descartado)

##Dataframes sin codigomes

### vehicular_db (descartado)

In [0]:
'''
vehicular_bd.describe()
'''

In [0]:
'''
#Eliminamos las filas duplicadas vehicular_bd
vehicular_bd = vehicular_bd.drop_duplicates()
print('shape: {}'.format(vehicular_bd.shape))
print('id_persona duplicados: {}'.format(vehicular_bd.id_persona.duplicated().sum()))
print('filas duplicadas: {}'.format(vehicular_bd.duplicated().sum()))
'''

#### ¿Qué personas tienen más registros?

In [0]:
'''
vehicular_bd[]
'''

In [0]:
'''
vehicular_bd[vehicular_bd.id_persona == 25468]
'''

####Tratando los missing values

In [0]:
'''
#Detectamos si hay missing values
vehicular_bd.isnull().sum()
'''

In [0]:
'''
#¿Qué id_personas tienen la mayor cantidad de valores nulos en veh_var1?
df_null = vehicular_bd[vehicular_bd.veh_var1.isnull()]
top_15 = Counter(df_null['id_persona']).most_common(15)
print(top_15)
'''

In [0]:
'''
#¿Qué id_personas tienen la mayor cantidad de valores nulos en veh_var2?
df_null = vehicular_bd[vehicular_bd.veh_var2.isnull()]
top_15 = Counter(df_null['id_persona']).most_common(15)
print(top_15)
'''


Por lo visto, la mejor forma de imputar los missing values será sacar dropear las columnas ya que no son muchas


In [0]:
'''
#Hacemos drop a los missing values de la columna Marca ya que son solo 56 registros
vehicular_bd = vehicular_bd.dropna(how='any')
'''

In [0]:
'''
#Comprobamos que no hay missing values
vehicular_bd.isnull().sum()
'''

In [0]:
'''
vehicular_bd.shape
#Se dropean 5,141 columnas (6.6% del df original)
'''

#### Creando tabla vehicular_bd con id_persona como primary key

In [0]:
'''
#Creampos una serie con la cuenta de las marcas 
seriex = vehicular_bd.groupby('id_persona')['marca'].agg('count')
#creamos un dataframe donde se promedian las variables numéricas para cada id_persona
dframex = vehicular_bd.groupby('id_persona')['veh_var1','veh_var2'].mean()
'''

In [0]:
'''
#Unimos seriex y dframex
vehicular_bd = pd.merge(seriex, dframex, on='id_persona').reset_index()
vehicular_bd.head()
'''

In [0]:
'''
vehicular_bd.marca.value_counts()
'''

###Sunat_bd

In [0]:
'''
#Detectamos si hay missing values
sunat_bd.isnull().sum()
'''

In [0]:
sunat_bdc = sunat_bd.groupby(['id_persona']).agg({'activ_econo':'count','meses_alta':'sum'}).reset_index()
#Se usa reset_index para que 'id_persona' no sea el index y siga siendo una columna
sunat_bdc.head()

,id_persona,activ_econo,meses_alta
0,1,1,273
1,3,1,30
2,5,1,46
3,6,1,307
4,9,1,159


In [0]:
'''
sunat_bd.activ_econo.value_counts()
'''

In [0]:
sunat_bdc.to_csv('sunat_bdc.csv', index=False)

###reniec_bd

In [0]:
'''
#Detectamos si hay missing values
reniec_bd.isnull().sum()
'''

In [0]:
'''
#Evaluamos los valores que hay en soc_var6
reniec_bd.soc_var6.unique()
'''

In [0]:
#Parece ser un feature categórico por lo que se llena los valores nulos con otra variable categótica (0)
reniec_bdc = reniec_bd.fillna(0)
reniec_bdc.isnull().sum()

id_persona    0
soc_var1      0
soc_var2      0
soc_var3      0
soc_var4      0
soc_var5      0
soc_var6      0
dtype: int64

In [0]:
reniec_bdc.to_csv('reniec_bdc.csv', index=False)

##Creando train y test dataset

###Merge las tablas temporales por prediction_id

In [0]:
#Leyendo train y test
train_bd = pd.read_csv('ib_base_inicial_train.csv', parse_dates=['codmes'], infer_datetime_format=True)
test_bd = pd.read_csv('ib_base_inicial_test.csv', parse_dates=['codmes'], infer_datetime_format=True)
#Leyendo tablas complementarias consulidadas
rcc_bd2 = pd.read_csv('rcc_bd2.csv')
digital_bdc = pd.read_csv('digital_bdc.csv')
reniec_bdc = pd.read_csv('reniec_bdc.csv')
sunat_bdc = pd.read_csv('sunat_bdc.csv') 

In [0]:
#Creando los lags mensuales 
#rcc
lag_rcc_1810 = rcc_bd2[rcc_bd2.codmes == 201810]
lag_rcc_1811 = rcc_bd2[rcc_bd2.codmes == 201811]
lag_rcc_1812 = rcc_bd2[rcc_bd2.codmes == 201812]
lag_rcc_1901 = rcc_bd2[rcc_bd2.codmes == 201901]
lag_rcc_1902 = rcc_bd2[rcc_bd2.codmes == 201902]
lag_rcc_1903 = rcc_bd2[rcc_bd2.codmes == 201903]

#digital
lag_dig_1811 = digital_bdc[digital_bdc.codmes == 201811]
lag_dig_1812 = digital_bdc[digital_bdc.codmes == 201812]
lag_dig_1901 = digital_bdc[digital_bdc.codmes == 201901]
lag_dig_1902 = digital_bdc[digital_bdc.codmes == 201902]
lag_dig_1903 = digital_bdc[digital_bdc.codmes == 201903]
lag_dig_1904 = digital_bdc[digital_bdc.codmes == 201904]

###Creando el train dataset


In [0]:
#MERGE CON DATAFRAMES ATEMPORALES
sdfs = [train_bd, reniec_bdc, sunat_bdc] #ORDENADO DESDE LOS DF CON MAS REGISTROS HASTA EL DE MENOS
train_bdc = reduce(lambda left,right: pd.merge(left,right,on='id_persona', how='left'), sdfs)
train_bdc['id_persona'] = train_bdc.id_persona.astype(str)
train_bdc.head(1)

,codmes,id_persona,codtarget,margen,cem,ingreso_neto,linea_ofrecida,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta
0,201901,165747,0,-5.0,1.0,1781.0,1700,0.773585,4.0,1.0,6.0,5.0,5.0,1.0,119.0


In [0]:
#Creando columna prediction_id
train_bdc['prediction_id'] = train_bdc.id_persona.apply(str) + '_' + train_bdc.codmes.apply(str)
train_bdc.head(1)

,codmes,id_persona,codtarget,margen,cem,ingreso_neto,linea_ofrecida,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta,prediction_id
0,201901,165747,0,-5.0,1.0,1781.0,1700,0.773585,4.0,1.0,6.0,5.0,5.0,1.0,119.0,165747_201901


In [0]:
#Separando train_bd en meses
#train_1901 = train_bdc[train_bdc.codmes == '201901']
train_1902 = train_bdc[train_bdc.codmes == '201902']
train_1903 = train_bdc[train_bdc.codmes == '201903']
train_1904 = train_bdc[train_bdc.codmes == '201904']

In [0]:
#Merge de trains con complementos (1 lag). Se omite train_1901 
#ya que no se tiene suficientes datos en tablas complementarias
sdfs = [train_1902, lag_rcc_1810, lag_dig_1811]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str) 
  #Se convierte en str de nuevo ya que al leer el csv se transforma en integer

train_1902 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
sdfs = [train_1903, lag_rcc_1811, lag_dig_1812]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str)

train_1903 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
sdfs = [train_1904,  lag_rcc_1812, lag_dig_1901]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str)

train_1904 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
#Concatenamos todos los train en un solo df
lista = [train_1902, train_1903, train_1904]
train_bdc = pd.concat(lista, ignore_index=True, sort=False) 
#Se usa sort para que las columnas nose desordenen

In [0]:
#Debido a que solo 6 personas no tiene soc_var, se dropean dichos registros
lista = ['soc_var1', 'soc_var2', 'soc_var3', 'soc_var4', 'soc_var5','soc_var6']
train_bdc = train_bdc.dropna(subset=lista,how='all')

In [0]:
#Se reemplazan todos los missing values con 0
train_bdc.fillna(0, inplace=True)

In [0]:
train_bdc.head()

,codmes,id_persona,codtarget,margen,cem,ingreso_neto,linea_ofrecida,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta,prediction_id,LINEA_TOTAL_TC,PRESTAMOS_COMERCIALES,MORA_PRESTAMOS_COMERCIALES,PRESTAMO_PERSONAL,MORA_PRESTAMO_PERSONAL,PRODUCTOS_BAJO_IMPACTO,MORA_PROD_BAJA_IMP,TARJETAS_COMPRAS,MORA_TARJETAS_COMPRAS,TARJETAS_EFECTIVO,MORA_TARJETAS_EFECTIVO,TARJ_OTROS_CONCEPTOS,MORA_TARJ_OTROS_CONCEPTOS,cod_banco,simu_prestamo,benefit,email,facebook,goog,youtb,compb,movil,desktop,n_sesion,n_pag,time_ctasimple,busqtc,busqvisa,busqcsimp,busqmill,busqcsld,busq,n_rep30,lima_dig,provincia_dig,extranjero_dig,iphone
0,201902,2651,0,-5.0,4507.0,9344.0,23300,0.547170,4.0,2.0,6.0,5.0,11.0,1.0,2.0,2651_201902,9417.000,0.0000,0.0,14778.2733,1.0,46339.5459,1.0,803.5840,1.0,6827.5440,1.0,0.4380,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,201902,5164,0,-5.0,310.0,3166.0,6300,0.528302,4.0,1.0,6.0,5.0,10.0,1.0,44.0,5164_201902,15380.589,8910.8618,1.0,0.0000,0.0,0.0000,0.0,151.5918,1.0,4542.1987,1.0,0.0000,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,201902,81364,0,-5.0,791.0,1870.0,1800,0.584906,4.0,2.0,1.0,2.0,9.0,0.0,0.0,81364_201902,365.000,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,170.3455,1.0,0.0000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,201902,206604,1,-5.0,531.0,2520.0,8800,0.867925,4.0,2.0,6.0,5.0,6.0,0.0,0.0,206604_201902,11680.000,0.0000,0.0,0.0000,0.0,0.0000,0.0,2401.9920,1.0,0.0000,0.0,687.3242,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201902,236904,0,-5.0,409.0,1955.0,3900,0.886792,4.0,2.0,6.0,5.0,3.0,0.0,0.0,236904_201902,17556.500,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
print('shape: {}'.format(train_bdc.shape))
print('prediction_id duplicados: {}'.format(train_bdc.prediction_id.duplicated().sum()))
print('filas duplicadas: {}'.format(train_bdc.duplicated().sum()))

shape: (158570, 53)
prediction_id duplicados: 0
filas duplicadas: 0


In [0]:
#Exportando tabla
train_bdc.to_csv('Dataset_train.csv', index=False)

###Creando el test dataset

In [0]:
test_bd = pd.read_csv('ib_base_inicial_test.csv', parse_dates=['codmes'], infer_datetime_format=True)

In [0]:
#MERGE CON DATAFRAMES ATEMPORALES
sdfs = [test_bd, reniec_bdc, sunat_bdc] #ORDENADO DESDE LOS DF CON MAS REGISTROS HASTA EL DE MENOS
test_bdc = reduce(lambda left,right: pd.merge(left,right,on='id_persona', how='left'), sdfs)
test_bdc['id_persona'] = test_bdc.id_persona.astype(str)
test_bdc.head(1)

,codmes,id_persona,cem,ingreso_neto,linea_ofrecida,prediction_id,soc_var1,soc_var2,soc_var3,soc_var4,soc_var5,soc_var6,activ_econo,meses_alta
0,201907,123381,1.0,1340.0,2600,123381_201907,0.679245,4.0,1.0,6.0,5.0,4.0,NaN,NaN


In [0]:
#Separando test_bd en meses
test_1905 = test_bdc[test_bdc.codmes == '201905']
test_1906 = test_bdc[test_bdc.codmes == '201906']
test_1907 = test_bdc[test_bdc.codmes == '201907']

In [0]:
#Merge de tests con complementos (1 lag). Se omite train_1901 
#ya que no se tiene suficientes datos en tablas complementarias
sdfs = [test_1905, lag_rcc_1901, lag_dig_1902]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str) 
  #Se convierte en str de nuevo ya que al leer el csv se transforma en integer

test_1905 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
#Merge de tests con complementos (1 lag). Se omite train_1901 
#ya que no se tiene suficientes datos en tablas complementarias
sdfs = [test_1906, lag_rcc_1902, lag_dig_1903]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str) 
  #Se convierte en str de nuevo ya que al leer el csv se transforma en integer

test_1906 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
#Merge de tests con complementos (1 lag). Se omite train_1901 
#ya que no se tiene suficientes datos en tablas complementarias
sdfs = [test_1907, lag_rcc_1903, lag_dig_1904]
for i in sdfs:
  i.id_persona = i.id_persona.astype(str) 
  #Se convierte en str de nuevo ya que al leer el csv se transforma en integer

test_1907 = reduce(lambda right, left: 
                    pd.merge(right, left.drop(['codmes','prediction_id'], 
                                              axis=1), on='id_persona', how='left'),sdfs)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
#Concatenamos todos los test en un solo df
lista = [test_1905, test_1906, test_1907]
test_bdc = pd.concat(lista, ignore_index=True, sort=False) 
#Se usa sort para que las columnas nose desordenen

In [0]:
#A pesar de que solo 23 personas no tiene soc_var, no se dropean dichos registros
#Se reemplazan todos los missing values con 0
test_bdc.fillna(0, inplace=True)

In [0]:
print('shape: {}'.format(test_bdc.shape))
print('id_persona duplicados: {}'.format(test_bdc.id_persona.duplicated().sum()))
print('prediction_id duplicados: {}'.format(test_bdc.prediction_id.duplicated().sum()))
print('filas duplicadas: {}'.format(test_bdc.duplicated().sum()))

shape: (178685, 51)
id_persona duplicados: 36069
prediction_id duplicados: 0
filas duplicadas: 0


In [0]:
#Exportando tabla
test_bdc.to_csv('Dataset_test.csv', index=False)

#Modeling

##  training 

Hacer:
- Cross validation -  para evitar overfitting
